In [10]:
__file__ = '/Users/hamel/github/docs-explore/custom/scratchpad.ipynb'

In [11]:
from nbconvert.preprocessors import Preprocessor
from nbconvert import MarkdownExporter, TemplateExporter
from nbconvert.preprocessors import TagRemovePreprocessor
import traitlets
from traitlets.config import Config
from pathlib import Path

In [12]:
from fastcore.all import L
from traitlets import Unicode
import re

In [13]:
re.sub('^\s*(%%|%).+?[\n\r]', '', '% foo hello \n %hi')

' %hi'

In [14]:
class TestMagic(Preprocessor):
    """A preprocessor to remove cell magic commands"""
    
    def preprocess_cell(self, cell, resources, index):
        if cell.cell_type == 'code':
            cleaned = L(cell.source.split('\n')).filter(lambda x: not x.strip().startswith('# @register'))
            cell.source = '\n'.join(cleaned).strip()
        return cell, resources

In [15]:
class CleanMagics(Preprocessor):
    """A preprocessor to remove cell magic commands"""
    pattern = '^\s*(%%|%).+?[\n\r]'
    
    def preprocess_cell(self, cell, resources, index):
        if cell.cell_type == 'code': cell.source = re.sub(self.pattern, '', cell.source).strip()
        return cell, resources

In [38]:
def get_exporter(template_file='ob.tpl'):
    c = Config()
    c.TagRemovePreprocessor.remove_cell_tags = ("remove_cell",)
    c.TagRemovePreprocessor.remove_all_outputs_tags = ('remove_output',)
    c.TagRemovePreprocessor.remove_input_tags = ('remove_input',)
    c.MarkdownExporter.preprocessors = [CleanMagics]
    tmp_dir = Path(__file__).parent/'templates/'
    tmp_file = tmp_dir/f"{template_file}"
    if not tmp_file.exists(): raise ValueError(f"{tmp_file} does not exist in {tmp_dir}")
    c.MarkdownExporter.template_file = str(tmp_file)
    return MarkdownExporter(config=c)

In [41]:
hamel = get_exporter()

In [42]:
print(hamel.from_filename('Custom_Doc_System.ipynb')[0])

---
title: my hello page title
description: my hello page description
hide_table_of_contents: true
---

```python
# from IPython.core.magic import register_cell_magic
# import tempfile, os

# @register_cell_magic
# def mf(line, cell):
#     with tempfile.NamedTemporaryFile() as fp:
#         cell_code = [s.encode('utf-8') for s in cell]
#         preamble = [b'from metaflow import *'] 
#         os.system(real_command)
```


```python
! echo hello
```

    hello



```python
from metaflow import FlowSpec, step

class MyFlow(FlowSpec):
    
    @step
    def start(self):
        print('this is the start')
        self.next(self.end)
    
    @step
    def end(self):
        print('this is the end')

if __name__ == '__main__':
    MyFlow()
```

    Overwriting myflow.py





    2





In [35]:
from jinja2 import DictLoader

dl = DictLoader({'ds':
"""
{%- extends 'markdown/index.md.j2' -%}

{% block input %}
```
This template works!
```
{% endblock input %}

"""})

In [37]:
# from jinja2 import DictLoader

# dl = DictLoader({'footer':
# """
# {%- extends 'lab/index.html.j2' -%}

# {% block footer %}
# FOOOOOOOOTEEEEER
# {% endblock footer %}
# """})


# exportHTML = HTMLExporter(extra_loaders=[dl], template_file='footer')
# (body, resources) = exportHTML.from_notebook_node(jake_notebook)
# for l in body.split('\n')[-4:]:
#     print(l)